In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, BasePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.output_parsers.xml import XMLOutputParser
from pydantic import BaseModel, Field

In [3]:
load_dotenv()

True

In [25]:
class CustomProduct(BaseModel):
    name: str = Field(description="The name of the product")
    price: float = Field(description="The price of the product")
    description: str = Field(description="A brief description of the product")

In [32]:
chat_openai = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. You will answer the user's question about a product in the format defined by pydantic class."),
        ("user", "{query}"),
    ]
)

prompt2 = PromptTemplate(
    template="You are a helpful assistant. Answer the user's question about a product in {format_instructions} as per user: {query}",
    input_variables=["query"],
    partial_variables={
        "format_instructions": CustomProduct.model_json_schema()["properties"]
    }
)

output_parser = JsonOutputParser(pydantic_object=CustomProduct)

## Way 1 - ChatPromptTemplate

In [28]:
chain = prompt | chat_openai | output_parser
chain

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a helpful assistant. You will answer the user's question about a product in the format defined by pydantic class."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10aa68b00>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10aa6a2c0>, root_client=<openai.OpenAI object at 0x10aa69220>, root_async_client=<openai.AsyncOpenAI object at 0x10aa690f0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| JsonOutputParser(pydantic_object=<class '__main__.CustomProduct'>)

In [29]:
response_json = chain.invoke("What is the price of Iphone 15?")

In [30]:
print(response_json)

{'product': 'iPhone 15', 'price': 'Available from $799'}


## Way 2 - PromptTemplate

In [33]:
chain2 = prompt2 | chat_openai
chain2

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': {'name': {'description': 'The name of the product', 'title': 'Name', 'type': 'string'}, 'price': {'description': 'The price of the product', 'title': 'Price', 'type': 'number'}, 'description': {'description': 'A brief description of the product', 'title': 'Description', 'type': 'string'}}}, template="You are a helpful assistant. Answer the user's question about a product in {format_instructions} as per user: {query}")
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10aa716a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10aa71e10>, root_client=<openai.OpenAI object at 0x10aa719d0>, root_async_client=<openai.AsyncOpenAI object at 0x10aa717b0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [34]:
response_json = chain.invoke("What is the price of Iphone 15?")


In [35]:
print(response_json)

{'product': 'iPhone 15', 'price': 'Starting at $799 for the base model'}
